In [1]:
!pip install google-api-python-client google-auth google-auth-oauthlib markdown

In [2]:
from google.colab import auth
auth.authenticate_user()

MessageError: Error: credential propagation was unsuccessful

In [3]:
from google.colab import auth
auth.authenticate_user()

In [10]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Create Docs API service
docs_service = build("docs", "v1")

# Create a new document
doc = docs_service.documents().create(body={"title": "Product Team Sync - Converted"}).execute()
doc_id = doc.get("documentId")
print("Created Document:", f"https://docs.google.com/document/d/{doc_id}/edit")

Created Document: https://docs.google.com/document/d/1oGwsyfpz5-JDV4cpwStL-_3O21ujNo7RN_u4s4KymV0/edit


In [11]:
# 1) Store the provided markdown as a string
markdown_text = """# Product Team Sync - May 15, 2023
## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)
## Agenda
### 1. Sprint Review
* Completed Features
 * User authentication flow
 * Dashboard redesign
 * Performance optimization
 * Reduced load time by 40%
 * Implemented caching solution
* Pending Items
 * Mobile responsive fixes
 * Beta testing feedback integration
### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
 * Navigation confusion
 * Color contrast issues
### 3. Next Sprint Planning
* Priority Features
 * Payment gateway integration
 * User profile enhancement
 * Analytics dashboard
* Technical Debt
 * Code refactoring
 * Documentation updates
## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal
## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders
## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets
---
Meeting recorded by: Sarah Chen
Duration: 45 minutes"""

In [12]:
# 2) Convert markdown_text into Google Docs API requests
#    (headings, bullets, checkboxes, @mentions, footer styling)

import re

requests = []
cursor = 1  # Google Docs indices start at 1


def add_paragraph(text,
                  style="NORMAL_TEXT",
                  bullet_type=None,
                  indent_level=0,
                  is_footer=False):
    """
    Insert one line of text as a paragraph and optionally:
    - apply heading style
    - turn it into a bullet or checkbox
    - indent it
    - style @mentions
    - style footer lines
    """
    global cursor, requests

    content = text + "\n"
    start = cursor
    end = start + len(content)

    # 1) Insert the text
    requests.append({
        "insertText": {
            "location": {"index": start},
            "text": content
        }
    })

    # 2) Paragraph style (heading, indent, footer alignment)
    paragraph_style = {"namedStyleType": style}
    fields = ["namedStyleType"]

    if indent_level > 0:
        paragraph_style["indentStart"] = {
            "magnitude": indent_level * 18,  # 18pt per level
            "unit": "PT"
        }
        fields.append("indentStart")

    if is_footer:
        paragraph_style["alignment"] = "END"
        fields.append("alignment")

    requests.append({
        "updateParagraphStyle": {
            "range": {"startIndex": start, "endIndex": end},
            "paragraphStyle": paragraph_style,
            "fields": ",".join(fields)
        }
    })

    # 3) Bullets or checkboxes
    if bullet_type == "BULLET":
        requests.append({
            "createParagraphBullets": {
                "range": {"startIndex": start, "endIndex": end},
                "bulletPreset": "BULLET_DISC_CIRCLE_SQUARE"
            }
        })
    elif bullet_type == "CHECKBOX":
        requests.append({
            "createParagraphBullets": {
                "range": {"startIndex": start, "endIndex": end},
                "bulletPreset": "BULLET_CHECKBOX"
            }
        })

    # 4) Style @mentions (bold + colored)
    for match in re.finditer(r"@\w+", text):
        m_start = start + match.start()
        m_end = start + match.end()
        requests.append({
            "updateTextStyle": {
                "range": {"startIndex": m_start, "endIndex": m_end},
                "textStyle": {
                    "bold": True,
                    "foregroundColor": {
                        "color": {"rgbColor": {"red": 0.1, "green": 0.4, "blue": 0.9}}
                    }
                },
                "fields": "bold,foregroundColor"
            }
        })

    # 5) Footer text italics
    if is_footer:
        requests.append({
            "updateTextStyle": {
                "range": {"startIndex": start, "endIndex": end},
                "textStyle": {"italic": True},
                "fields": "italic"
            }
        })

    cursor = end


# Build all requests from the markdown text
for raw_line in markdown_text.split("\n"):
    line = raw_line.rstrip("\n")
    stripped = line.strip()

    # Blank line → just newline paragraph
    if stripped == "":
        add_paragraph("", style="NORMAL_TEXT")
        continue

    # Horizontal rule '---' → separator blank line
    if stripped == "---":
        add_paragraph("", style="NORMAL_TEXT")
        continue

    # Footer detection
    is_footer = (
        stripped.startswith("Meeting recorded by:") or
        stripped.startswith("Duration:")
    )

    # Headings
    if stripped.startswith("# "):
        add_paragraph(stripped[2:], style="HEADING_1")
        continue
    elif stripped.startswith("## "):
        add_paragraph(stripped[3:], style="HEADING_2")
        continue
    elif stripped.startswith("### "):
        add_paragraph(stripped[4:], style="HEADING_3")
        continue

    # Leading spaces → indent level (for nested bullets)
    leading_spaces = len(line) - len(line.lstrip(" "))
    indent_level = leading_spaces  # simple mapping; enough for this doc

    # Checkboxes: "- [ ] ..."
    if stripped.startswith("- [ ]"):
        text = stripped[5:].strip()
        add_paragraph(text,
                      style="NORMAL_TEXT",
                      bullet_type="CHECKBOX",
                      indent_level=indent_level)
        continue

    # Bullet lists: "* ..." or "- ..."
    if stripped.startswith("* ") or stripped.startswith("- "):
        text = stripped[2:].strip()
        add_paragraph(text,
                      style="NORMAL_TEXT",
                      bullet_type="BULLET",
                      indent_level=indent_level)
        continue

    # Default: normal paragraph (for non-list lines like footer)
    add_paragraph(stripped, style="NORMAL_TEXT", is_footer=is_footer)

len(requests)



135

In [13]:
from googleapiclient.errors import HttpError

try:
    result = docs_service.documents().batchUpdate(
        documentId=doc_id,
        body={"requests": requests}
    ).execute()
    print("Document updated successfully!")
    print("Link:", f"https://docs.google.com/document/d/{doc_id}/edit")
except HttpError as e:
    print("Error while updating document:", e)

Document updated successfully!
Link: https://docs.google.com/document/d/1oGwsyfpz5-JDV4cpwStL-_3O21ujNo7RN_u4s4KymV0/edit
